In [1]:
%matplotlib widget

import matplotlib.pyplot as plt
import rospy
import tf
from nav_msgs.msg import Odometry
from tf.transformations import quaternion_matrix
import numpy as np
from matplotlib.animation import FuncAnimation
from sensor_msgs.msg import LaserScan


In [2]:
import ipywidgets as widgets
import jupyros as jr
import os
from ipywidgets import *
from IPython.display import *
from std_msgs.msg import String
from assignment_3.srv import *
#from ipywidgets import Button, GridBox, Layout, ButtonStyle
from geometry_msgs.msg import Twist, Vector3 #for cmd_vel topic
from jupyros import ros3d

reach_task = []

In [3]:
#all the wigets
coo_x = widgets.FloatText(
        value=0.0,
        description='X coordinate:',
        disabled=False
    )

coo_y = widgets.FloatText(
    value=0.0,
    description='Y coordinate:',
    disabled=False
    )

button1 = widgets.Button(
    description='Reach coordinate',
    disabled=False,
    )

button2 = widgets.Button(
    description='Drive robot',
    disabled=False,
    )

button3 =  widgets.Button(
    description='Start',
    disabled=False,
    )

button4 = widgets.Button(
    description='Undo',
    disabled=False,
    )

buttonUP = widgets.Button(
    #description='UP',
    disabled=False,
    icon='angle-up', # (FontAwesome names without the `fa-` prefix)
    )

buttonDOWN = widgets.Button(
    #description='DOWN',
    disabled=False,
    icon='angle-down' # (FontAwesome names without the `fa-` prefix)
    )

buttonRIGHT = widgets.Button(
    #description='RIGHT',
    disabled=False,
    icon='angle-right' # (FontAwesome names without the `fa-` prefix)
    )

buttonLEFT = widgets.Button(
    #description='LEFT',
    disabled=False,
    icon='angle-left' # (FontAwesome names without the `fa-` prefix)
    )

buttonPAUSE = widgets.Button(
    #description='PAUSE',
    disabled=False,
    icon='pause' # (FontAwesome names without the `fa-` prefix)
    )

buttonUPLEFT = widgets.Button(
    description='UPLEFT',
    disabled=False,
    )

buttonDOWNLEFT = widgets.Button(
    description='DOWNLEFT',
    disabled=False,
    )

buttonUPRIGHT = widgets.Button(
    description='UPRIGHT',
    disabled=False,
    )

buttonDOWNRIGHT = widgets.Button(
    description='DOWNRIGHT',
    disabled=False,
    )

selection = widgets.SelectionSlider(
    options=['off', 'on'],
    value='off',
    description='Need some help',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    style = {'description_width': 'initial'}
    )

ghost = widgets.Label(
    value=""
    )

box_arrow = widgets.HBox(
    [widgets.VBox([buttonUPLEFT, buttonLEFT, buttonDOWNLEFT]),
     widgets.VBox([buttonUP, buttonPAUSE, buttonDOWN]),
     widgets.VBox([buttonUPRIGHT, buttonRIGHT, buttonDOWNRIGHT])]
    )

box_menu = widgets.HBox(
    [widgets.VBox([box_arrow]),
     widgets.VBox([ghost,ghost, ghost]),
     widgets.VBox([selection, ghost, button4])]
    )


In [4]:
def on_value_change_selection(change):
    #print("here")
    if selection.value == "on":
        optn = 1
    else:
        optn = 0
        
    rospy.set_param("/selection_param", optn)

In [5]:
def set_cmd_vel(drc):
    v_init = Vector3(0,0,0)
    vel = Twist(v_init,v_init)
    
    const = 0.5
    
    if drc == 1:
        vel.linear.x = const
        vel.angular.z = const
    elif drc == 2:
        vel.linear.x = const
    elif drc == 3:
        vel.linear.x = const
        vel.angular.z = -const
    elif drc == 4:
        vel.angular.z = const
    elif drc == 6:
        vel.angular.z = -const
    elif drc == 7:
        vel.linear.x = -const
        vel.angular.z = const
    elif drc == 8:
        vel.linear.x = -const
    elif drc == 9:
        vel.linear.x = -const
        vel.angular.z = -const
        
    #print("here")
        
    pub = rospy.Publisher('/remap_cmd_vel', Twist, queue_size=10)
    pub.publish(vel)
      

In [6]:
def bu(bu):
    set_cmd_vel(2)

def bul(bul):
    set_cmd_vel(1)
    
def bur(bur):
    set_cmd_vel(3)
    
def bd(bd):
    set_cmd_vel(8)

def bdl(bdl):
    set_cmd_vel(7)
    
def bdr(bdr):
    set_cmd_vel(9)
    
def bl(bl):
    set_cmd_vel(4)
    
def br(br):
    set_cmd_vel(6)

def bp(bp):
    set_cmd_vel(5)

In [7]:
def display_map():
    
    v = ros3d.Viewer()
    rc = ros3d.ROSConnection(url="ws://localhost:9090")
    tf_client = ros3d.TFClient(ros=rc, fixed_frame='map')
    laser_view = ros3d.LaserScan(topic="/scan", ros=rc, tf_client=tf_client)
    map_view = ros3d.OccupancyGrid(topic="/map", ros=rc, tf_client=tf_client, continuous=True)
    path = ros3d.Path(topic="/move_base/NavfnROS/plan", ros=rc,
    tf_client=tf_client)
    urdf = ros3d.URDFModel(ros=rc, tf_client=tf_client,
    path=os.environ.get('JUPYROS_ASSETS_URL', 'http://localhost:3000'))
    g = ros3d.GridModel()
    #v.objects = [g, laser_view, map_view, path, urdf]
    v.objects = [laser_view, map_view, urdf]
    
    display(v)

In [8]:
def drive_interface():
    #print("here")

    display(box_menu)
    display_map()
    
    #undo botton
    button4.on_click(obc_4)
    
    buttonUP.on_click(bu)
    buttonUPLEFT.on_click(bul)
    buttonUPRIGHT.on_click(bur)
    buttonDOWN.on_click(bd)
    buttonDOWNLEFT.on_click(bdl)
    buttonDOWNRIGHT.on_click(bdr)
    buttonLEFT.on_click(bl)
    buttonRIGHT.on_click(br)
    buttonPAUSE.on_click(bp)
    

In [9]:
def obc_3(obc):
    global reach_task
    
    button3.disabled=True
    rospy.wait_for_service("/reachCoordinateService")
    client = rospy.ServiceProxy("/reachCoordinateService", reachCoordinateService)
    #store and interpretation of the return
    rt = client(coo_x.value,coo_y.value)
    if rt.ret == 0:
        reach_task.append("not reached")
        print("Target not reached, goal cancelled")
    elif rt.ret == 1:
        reach_task.append("reached")
        print("Target reached!")
        
def obc_4(obc):
    clear_output()
    button2.disabled=False
    button1.disabled=False
    button3.disabled=False
    display(button1,button2)


In [10]:
def obc_1(obc):
    #print("Button1 clicked.")
    
    #Disable the button so the user can't reclick it again
    button1.disabled=True
    button2.disabled=True
    
    display(coo_x,coo_y)
    #display_map()
    
    display(button3,button4)
    
    button3.on_click(obc_3)
    button4.on_click(obc_4)
    

def obc_2(obc):
    #print("Button2 clicked.")
    
    #Disable the button so the user can't reclick it again
    button2.disabled=True
    button1.disabled=True
    drive_interface()

In [11]:
class Visualiser:
    
    counter = 250
    
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.ln, = plt.plot([], [], 'r.')
        plt.xlabel('x coordinate')
        plt.ylabel('y coordinate')
        plt.title('Path of the robot')
        self.x_data, self.y_data = [] , []
        
    def plot_init(self):
        self.ax.set_xlim(-10, 10)
        self.ax.set_ylim(-10, 10)
        return self.ln
    
    def odom_callback(self, msg):
       
        self.counter= self.counter -1
    
        if self.counter == 0:
            if len(self.y_data) < 10000:
                self.y_data.append(msg.pose.pose.position.y)
                self.x_data.append(msg.pose.pose.position.x)
            else:
                self.y_data.pop(0)
                self.x_data.pop(0)
                self.y_data.append(msg.pose.pose.position.y)
                self.x_data.append(msg.pose.pose.position.x)
            self.counter = 250
            
    def update_plot(self, frame):
        self.ln.set_data(self.x_data, self.y_data)
        return self.ln
    
class VisualiserScan:
    
    def __init__(self):
        self.fig, self.ax = plt.subplots(subplot_kw={'projection': 'polar'})
        self.ln, = plt.plot([], [], 'r.')
        plt.title('Visualization of the scanner')
        self.theta_data = np.arange(0, np.pi, np.pi/720)
        self.d_data = []
        
    def plot_init(self):
        self.ax.set_rmax(15)
        return self.ln
    
    def laser_callback(self, msg):
        self.d_data = msg.ranges
        
    def update_plot(self, frame):
        self.ln.set_data(self.theta_data, self.d_data)
        return self.ln

rospy.init_node('jn_user_interaction')
print("What do you want to do?")
    
display(button1,button2)

button1.on_click(obc_1)
button2.on_click(obc_2)


What do you want to do?


Button(description='Reach coordinate', style=ButtonStyle())

Button(description='Drive robot', style=ButtonStyle())

Viewer(objects=[LaserScan(ros=ROSConnection(url='ws://localhost:9090'), tf_client=TFClient(fixed_frame='map', …

In [12]:
vis = Visualiser()
sub = rospy.Subscriber('/odom', Odometry, vis.odom_callback)
ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init)
plt.show(block=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
vis2 = VisualiserScan()
sub2 = rospy.Subscriber('/scan', LaserScan , vis2.laser_callback)
ani2 = FuncAnimation(vis2.fig, vis2.update_plot, init_func=vis2.plot_init)
plt.show(block=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
fig, ax = plt.subplots()
_, _, ln, = ax.hist([])

def init():
    plt.ylabel('# of goal')
    plt.title('Reached and NON-reached targets')
    return ln,

def update(frame):
    ax.hist(reach_task,bins=10,color='green')
    return ln,

ani3 = FuncAnimation(fig, update, init_func=init, blit=False)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …